# Python tutorial: „lazy“ iteratoriai duomenų analitikoje (detaliau)

## Tikslas

- Detaliai paaiškinti „lazy“ (tingaus) skaičiavimo esmę Python kalboje.
- Pademonstruoti iteratorius su `range`, `zip`, `enumerate`, `map`, `filter`, generatoriais ir failų skaitymu.
- Pritaikyti `def` ir `lambda`, kai tai padidina aiškumą arba lankstumą.
- Pateikti verslo pavyzdžius: filtravimas, transformacija, suvestinės, kokybės kontrolė, didelių failų apdorojimas.
- Paminėti dažniausias klaidas ir tipines situacijas, kurios sukelia netikėtą elgesį.

## 1) „Lazy“ esmė

„Lazy“ reiškia, kad rezultatai neskaičiuojami iš karto. Vietoje to sukuriamas iteratorius, kuris reikšmes pateikia po vieną, tik iteruojant.

Privalumai:
- mažesnės atminties sąnaudos,
- galimybė apdoroti didelius duomenų srautus,
- patogu jungti kelis apdorojimo žingsnius (pipeline).

Dažni „lazy“ objektai:
- `range`, `zip`, `enumerate`,
- `map`, `filter`,
- generatoriai ir generator expressions,
- failo objektas, iteruojant eilutėmis.

In [1]:
nums_list = [1, 2, 3]
nums_range = range(1, 4)

print("List:", nums_list)
print("Range:", nums_range)
print("Range -> list:", list(nums_range))

List: [1, 2, 3]
Range: range(1, 4)
Range -> list: [1, 2, 3]


## 2) Iteratorius ir „išsekimas“

Iteratorius yra objektas, kuris grąžina reikšmes po vieną. Panaudojus visas reikšmes, iteratorius „išsenka“.

Tai ypač svarbu su `map`, `filter`, `zip`, `enumerate` ir generatoriais.

In [5]:
it = iter([10, 20, 30])

print(next(it))
print(next(it))
print(next(it))

# Dabar iteratorius išseko. Ketvirtas kvietimas sukeltų StopIteration.
#print(next(it))

10
20
30


In [6]:
x = map(int, ["1", "2", "3"])
print("Pirmas list(x):", list(x))
print("Antras list(x):", list(x))  # antrą kartą jau tuščia

Pirmas list(x): [1, 2, 3]
Antras list(x): []


## 3) `range()` ir ciklai

`range()` yra „lazy“ seka, skirta iteracijai. Ji nenaudoja atminties kaip pilnas sąrašas.

In [ ]:
for i in range(1, 6):
    print(i)

## 4) `enumerate()` – indeksas + reikšmė

`enumerate()` dažnai naudojamas ataskaitoms numeruoti, klaidų eilutėms identifikuoti, eilučių tvarkai fiksuoti.

In [ ]:
products = ["Milk", "Bread", "Eggs"]

for i, p in enumerate(products, start=1):
    print(i, p)

### Verslo pavyzdys: klaidų sąrašas su eilučių numeriais

Situacija:
- turėti „nešvarias“ įvestis,
- pažymėti netinkamas eilutes su numeriu,
- palikti tvarkingus įrašus.

In [ ]:
raw_qty = ["2", "x", "3", "", "5"]

good = []
bad = []

for row_no, text in enumerate(raw_qty, start=1):
    try:
        qty = int(text)
        if qty <= 0:
            raise ValueError("qty turi būti teigiamas")
        good.append(qty)
    except ValueError:
        bad.append({"row_no": row_no, "value": text})

print("Geri įrašai:", good)
print("Blogi įrašai:", bad)

## 5) `zip()` – sujungimas pagal poziciją

`zip()` sujungia kelias sekas pagal poziciją ir grąžina iteratorių.

Dažna praktika:
- sujungti stulpelius (pvz., produktas + kiekis),
- formuoti įrašus (records),
- sukurti žodyną iš dviejų sąrašų.

In [ ]:
names = ["Milk", "Bread", "Eggs"]
prices = [1.50, 0.99, 0.80]

z = zip(names, prices)
print(z)
print(list(z))

### Dažna klaida: `zip()` sustoja ties trumpiausiu sąrašu

Jei sąrašų ilgiai nesutampa, dalis duomenų gali būti „prarasta“ be klaidos.

In [ ]:
names2 = ["Milk", "Bread", "Eggs", "Butter"]
prices2 = [1.50, 0.99, 0.80]

if len(names2) != len(prices2):
    print("Įspėjimas: sąrašų ilgiai nesutampa. zip() apdoros tik dalį eilučių.")

print(list(zip(names2, prices2)))

### Verslo pavyzdys: krepšelio sumos skaičiavimas

- sujungti produktus ir kiekius per `zip()`,
- kainą paimti iš žodyno,
- suskaičiuoti eilučių sumas ir bendrą sumą.

In [ ]:
products = ["Milk", "Bread", "Eggs"]
qtys = [2, 1, 3]
price_map = {"Milk": 1.50, "Bread": 0.99, "Eggs": 0.80}

total = 0.0
lines = []

for product, qty in zip(products, qtys):
    price = price_map.get(product, 0.0)
    line_total = qty * price
    total += line_total
    lines.append((product, qty, price, round(line_total, 2)))

print("Eilutės:", lines)
print("Iš viso:", round(total, 2))

## 6) `map()` – transformacija

`map(function, iterable)` grąžina iteratorių, kuris pateikia transformuotus elementus.
`map()` yra „lazy“ ir rezultatai gaunami tik iteruojant.

In [ ]:
def to_int_safe(x):
    return int(x)

data = ["1", "2", "3"]
mapped = map(to_int_safe, data)

print(mapped)
print(list(mapped))

### `map()` su `lambda`

`lambda` patogu, kai funkcija trumpa ir naudojama tik vienoje vietoje.

In [ ]:
amounts = [100, 80, 40]
with_vat = map(lambda x: round(x * 1.21, 2), amounts)
print(list(with_vat))

### Verslo pavyzdys: normalizuoti tekstą

- pašalinti tarpus,
- paversti į mažąsias raides,
- palikti tvarkingą sąrašą.

In [ ]:
def normalize_text(s):
    return s.strip().lower()

raw_channels = [" Online ", "DIRECT", " partner "]
norm = map(normalize_text, raw_channels)
print(list(norm))

## 7) `filter()` – filtravimas

`filter(function, iterable)` grąžina iteratorių, kuris palieka elementus, atitinkančius sąlygą.

In [ ]:
def is_positive(x):
    return x > 0

values = [10, -5, 0, 20, 3]
filtered = filter(is_positive, values)
print(list(filtered))

### `filter()` su `lambda`

- palikti tik pajamas virš 100,
- parodyti tipinę analizės atranką.

In [ ]:
revenues = [50, 120, 90, 300, 110]
high = filter(lambda r: r > 100, revenues)
print(list(high))

### Svarbi klaida: `filter(None, ...)` pašalina ir 0 reikšmes

Jei 0 yra validi reikšmė (pvz., nuolaida 0.0), tokia filtracija netinka.

In [ ]:
discounts = [5, 0, 10, None, 0, 15]

print("filter(None, ...):", list(filter(None, discounts)))

clean = []
for d in discounts:
    if d is not None:
        clean.append(d)

print("Tik be None:", clean)

## 8) Generatoriai: `yield` ir generator expression

Generatorius yra funkcija su `yield`, kuri grąžina iteratorių.
Reikšmės generuojamos palaipsniui.

In [7]:
def running_total(values):
    total = 0
    for v in values:
        total += v
        yield total

daily_revenue = [100, 80, 40, 60]
rt = running_total(daily_revenue)

print(rt)
print(list(rt))

<generator object running_total at 0x0000017000D598C0>
[100, 180, 220, 280]


### Verslo pavyzdys: srautinė KPI suvestinė

- turėti dienos pajamas,
- gauti kaupiamąją sumą (YTD logikos principas),
- nekurti didelių tarpinių struktūrų.

In [ ]:
daily_revenue = [100, 80, 40, 60, 20]

for day_no, cum in enumerate(running_total(daily_revenue), start=1):
    print("Diena:", day_no, "Kaupiamoji suma:", cum)

### Generator expression su `sum()`

Generator expression `(expr for ...)` dažnai naudojamas greitam sumavimui be tarpinio sąrašo.

In [ ]:
lines = [
    {"qty": 2, "price": 1.50},
    {"qty": 1, "price": 0.99},
    {"qty": 3, "price": 0.80},
]

total = sum(line["qty"] * line["price"] for line in lines)
print(round(total, 2))

## 9) „Pipeline“: keli „lazy“ žingsniai iš eilės

Situacija:
- turėti mišrų sąrašą su tekstais,
- išvalyti reikšmes,
- palikti tik leidžiamus kanalus,
- gauti galutinį sąrašą materializuojant tik pabaigoje.

In [ ]:
raw = [" Online ", None, "", "DIRECT", " partner ", "phone", "online"]

def is_clean_str(x):
    return isinstance(x, str) and x.strip() != ""

def normalize_channel(x):
    return x.strip().lower()

allowed = {"online", "direct", "partner"}

step1 = filter(is_clean_str, raw)
step2 = map(normalize_channel, step1)
step3 = filter(lambda x: x in allowed, step2)

result = list(step3)
result

## 10) Failo skaitymas: natūralus „lazy“ apdorojimas

Failas iteruojamas eilutėmis, todėl galima apdoroti didelius failus be viso turinio krovimo į atmintį.
Demonstracijoje sukuriamas mažas failas.

In [ ]:
with open("demo_orders.csv", "w", encoding="utf-8") as f:
    f.write("order_id,revenue\n")
    f.write("1,100\n")
    f.write("2,80\n")
    f.write("3,40\n")
    f.write("4,120\n")

with open("demo_orders.csv", "r", encoding="utf-8") as f:
    header = next(f).strip()
    print("Antraštė:", header)

    total = 0
    high_orders = []

    for line_no, line in enumerate(f, start=2):
        parts = line.strip().split(",")
        try:
            revenue = int(parts[1])
        except ValueError:
            revenue = 0

        total += revenue
        if revenue > 100:
            high_orders.append({"line_no": line_no, "revenue": revenue})

print("Pajamos iš viso:", total)
print("Didelės pajamos:", high_orders)

## 11) Dažniausios klaidos dirbant su „lazy“ iteratoriais

- Tikėtis, kad `print(map_obj)` ar `print(zip_obj)` parodys reikšmes; dažniausiai bus rodomas tik objekto tipas.
- Materializuoti iteratorių su `list(...)` ir bandyti iteruoti dar kartą; iteratorius bus „išnaudotas“.
- Naudoti `zip()` su nevienodais ilgio sąrašais ir netyčia prarasti dalį eilučių.
- Naudoti `filter(None, ...)` ir netyčia pašalinti 0 reikšmes, nors jos gali būti validžios.
- Kurti `list(...)` labai dideliems srautams, kai pakanka iteruoti po vieną.

## Santrauka

- „Lazy“ iteratoriai pateikia reikšmes tik iteruojant, todėl taupo atmintį ir leidžia kurti apdorojimo grandines.
- `map` ir `filter` yra patogūs transformacijai ir filtravimui, ypač su „pipeline“ logika.
- Generatoriai su `yield` leidžia kurti srautinius skaičiavimus (pvz., kaupiamosios sumos).
- Failų skaitymas eilutėmis yra natūralus „lazy“ pavyzdys, dažnas analitikoje.